In [ ]:
#! Python3
# -*- coding: utf-8 -*-
# 異分野共著　process II
# 4. 全著者IDそれぞれに専門分野を割り当てる

import pandas as pd
import requests
import xml.etree.ElementTree as ET
import urllib
from urllib.error import URLError, HTTPError

# API Key   
myAPIkey = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

# 調べたい Author 
uniqueaIds = pd.read_csv('UniqueAuthorIdsList.csv', dtype= 'object')
uniqueaIds['ID'] = "'" + uniqueaIds['Author ID'] + "'"

# APIのひな型
api =  'https://api.elsevier.com/content/author/author_id/{sid}?apiKey={key}'
author_asjc27 = []
#4-1. 著者IDから分野（ASJC, 27分野） codeを抽出（API）
for i, number in uniqueaIds.iterrows():
        # APIのURLを得る
        aid = number['Author ID']
        url = api.format(sid=aid, key=myAPIkey)
        print('Scopus Author ID'+ str(i) + ' : ' + str(aid) )

        # 実際にAPIにリクエストを送信して結果を取得する
        try:
            req = urllib.request.Request(url)
            with urllib.request.urlopen(req) as response:
                r = requests.get(url)
        except URLError as e:
            author27 = [aid, 0, 0, 0]
            print(author27)
            author_asjc27.append(author27)
            if hasattr(e, 'reason'):
                print('Reason: ', e.reason)
                #APIkeyの制限超えたらリストを保存して止まる
                if e.reason == 'Too Many Requests':
                    break
            elif hasattr(e, 'code'):
                print('Error code: ', e.code)
        else:
            text = r.text.encode('utf-8')
            data = ET.fromstring(text)

            # 結果を出力 (Author id, subject-areas)
            #  Defined Author ID
            author_id = data[0][1].text.replace('AUTHOR_ID:','')
            
            # 論文数が5本以下ならスキップ
            try:
                dc = int(data.find('.//document-count').text)
            except:
                dc = 0
            
            if dc < 5:
                asjc27 = 0
                author27 = [aid, author_id, dc, asjc27]
            else:
                #author-profileの下から、分野のコードcode(4桁のうち上位2桁)とその頻度frequencyを抽出
                asjc = []
                asjc_list = []
                for et in data.findall('.//author-profile/classificationgroup/classifications[@type="ASJC"]/classification'):
                    code = int(et.text[:2])
                    freqs = int(et.get('frequency'))
                    asjc = [code, freqs]
                    asjc_list.append(asjc)

                #4-2. 一番多い分野を特定, 著者の分野を決定
                asjcdf = pd.DataFrame(asjc_list, columns=['code2', 'freqs'])
                asjcsort = asjcdf.groupby('code2',as_index=False)['freqs'].sum()
                asjcsort.sort_values(by='freqs', ascending=False, inplace = True)
                asjc27 = asjcsort.iat[0,0]
                author27 = [aid, author_id, dc, asjc27]
            print(author27)
            author_asjc27.append(author27)
    
#4-4. listを作成
author_asjc27_df = pd.DataFrame(author_asjc27, columns=['Scopus Author ID', 'responce ID', 'document count', 'asjc27'])
author_asjc27_df.to_csv('AuthorIDwithasjc27List.csv', index = False)


Scopus Author ID52626 : 6701754861
['6701754861', '6701754861', 186, 27]
Scopus Author ID52627 : 7004872600
['7004872600', '7004872600', 377, 27]
Scopus Author ID52628 : 7005329283
['7005329283', '7005329283', 613, 27]
Scopus Author ID52629 : 7006483248
['7006483248', '7006483248', 334, 27]
Scopus Author ID52630 : 7006862228
['7006862228', '7006862228', 134, 27]
Scopus Author ID52631 : 7102284091
['7102284091', '7102284091', 753, 27]
Scopus Author ID52632 : 7201984627
['7201984627', '7201984627', 246, 27]
Scopus Author ID52633 : 7202160748
['7202160748', '7202160748', 486, 27]
Scopus Author ID52634 : 7402229981
['7402229981', '7402229981', 277, 27]
Scopus Author ID52635 : 7403032447
['7403032447', '7403032447', 133, 27]
Scopus Author ID52636 : 8948013100
['8948013100', '8948013100', 78, 27]
Scopus Author ID52637 : 24358370000
['24358370000', '24358370000', 110, 31]
Scopus Author ID52638 : 35269208100
['35269208100', '35269208100', 64, 10]
Scopus Author ID52639 : 55704243000
['557042430

['36751064900', '36751064900', 20, 27]
Scopus Author ID52738 : 55340888500
['55340888500', '55340888500', 22, 27]
Scopus Author ID52739 : 56459586800
['56459586800', '56459586800', 5, 27]
Scopus Author ID52740 : 7003851710
['7003851710', '7003851710', 237, 27]
Scopus Author ID52741 : 7005311202
['7005311202', '7005311202', 33, 27]
Scopus Author ID52742 : 26640445300
['26640445300', '26640445300', 16, 27]
Scopus Author ID52743 : 40261243600
['40261243600', '40261243600', 14, 27]
Scopus Author ID52744 : 55487818000
['55487818000', '55487818000', 22, 27]
Scopus Author ID52745 : 55624479908
['55624479908', '55624479908', 28, 27]
Scopus Author ID52746 : 55628587272
['55628587272', '55628587272', 9, 27]
Scopus Author ID52747 : 6701780922
['6701780922', '6701780922', 71, 27]
Scopus Author ID52748 : 7402217769
['7402217769', '7402217769', 26, 13]
Scopus Author ID52749 : 55343953900
['55343953900', '55343953900', 3, 0]
Scopus Author ID52750 : 57116972600
['57116972600', '57116972600', 4, 0]
Sco

['57192435885', '57192435885', 14, 22]
Scopus Author ID52850 : 57194684898
['57194684898', '57194684898', 6, 25]
Scopus Author ID52851 : 35426128700
['35426128700', '35426128700', 242, 16]
Scopus Author ID52852 : 57194690235
['57194690235', '57194690235', 4, 0]
Scopus Author ID52853 : 7006725954
['7006725954', '7006725954', 368, 16]
Scopus Author ID52854 : 55173596300
['55173596300', '55173596300', 17, 19]
Scopus Author ID52855 : 57194201126
['57194201126', '57194201126', 1, 0]
Scopus Author ID52856 : 6602600408
['6602600408', '6602600408', 90, 19]
Scopus Author ID52857 : 10340439400
['10340439400', '10340439400', 43, 27]
Scopus Author ID52858 : 24178088800
['24178088800', '24178088800', 2, 0]
Scopus Author ID52859 : 26643936200
['26643936200', '26643936200', 127, 27]
Scopus Author ID52860 : 34769387900
['34769387900', '34769387900', 258, 13]
Scopus Author ID52861 : 35603395400
['35603395400', '35603395400', 109, 27]
Scopus Author ID52862 : 54783061200
['54783061200', '54783061200', 37

['24780309700', '24780309700', 17, 27]
Scopus Author ID52961 : 55578614800
['55578614800', '55578614800', 28, 13]
Scopus Author ID52962 : 55605449900
['55605449900', '55605449900', 11, 13]
Scopus Author ID52963 : 55960829700
['55960829700', '55960829700', 9, 13]
Scopus Author ID52964 : 56932272600
['56932272600', '56932272600', 3, 0]
Scopus Author ID52965 : 57188949260
['57188949260', '57188949260', 12, 13]
Scopus Author ID52966 : 57190021790
['57190021790', '57190021790', 1, 0]
Scopus Author ID52967 : 6603239610
['6603239610', '6603239610', 78, 13]
Scopus Author ID52968 : 7801535855
['7801535855', '7801535855', 52, 13]
Scopus Author ID52969 : 8619187600
['8619187600', '8619187600', 121, 27]
Scopus Author ID52970 : 8855613000
['8855613000', '8855613000', 22, 13]
Scopus Author ID52971 : 37011005700
['37011005700', '37011005700', 24, 22]
Scopus Author ID52972 : 54782773900
['54782773900', '54782773900', 16, 23]
Scopus Author ID52973 : 56083242900
['56083242900', '56083242900', 9, 22]
Sco

['35407026100', '35407026100', 597, 27]
Scopus Author ID53072 : 36182774600
['36182774600', '36182774600', 8, 27]
Scopus Author ID53073 : 36880080500
['36880080500', '36880080500', 22, 13]
Scopus Author ID53074 : 36914415200
['36914415200', '36914415200', 59, 27]
Scopus Author ID53075 : 54885267500
['54885267500', '54885267500', 14, 27]
Scopus Author ID53076 : 55104730600
['55104730600', '55104730600', 317, 27]
Scopus Author ID53077 : 55318773000
['55318773000', '55318773000', 4, 0]
Scopus Author ID53078 : 55863462000
['55863462000', '55863462000', 8, 13]
Scopus Author ID53079 : 56294178100
['56294178100', '56294178100', 6, 27]
Scopus Author ID53080 : 57191899001
['57191899001', '57191899001', 1, 0]
Scopus Author ID53081 : 57191899994
['57191899994', '57191899994', 1, 0]
Scopus Author ID53082 : 57191905575
['57191905575', '57191905575', 1, 0]
Scopus Author ID53083 : 6507207483
['6507207483', '6507207483', 55, 27]
Scopus Author ID53084 : 6507453612
['6507453612', '6507453612', 94, 27]
S

['7003555031', '7003555031', 59, 31]
Scopus Author ID53183 : 7004514138
['7004514138', '7004514138', 532, 31]
Scopus Author ID53184 : 7005736796
['7005736796', '7005736796', 422, 31]
Scopus Author ID53185 : 7402068004
['7402068004', '7402068004', 293, 31]
Scopus Author ID53186 : 15046253500
['15046253500', '15046253500', 22, 16]
Scopus Author ID53187 : 23025104200
['23025104200', '23025104200', 30, 30]
Scopus Author ID53188 : 54960763600
['54960763600', '54960763600', 18, 15]
Scopus Author ID53189 : 55875550000
['55875550000', '55875550000', 3, 0]
Scopus Author ID53190 : 56374470900
['56374470900', '56374470900', 5, 15]
Scopus Author ID53191 : 57188551196
['57188551196', '57188551196', 4, 0]
Scopus Author ID53192 : 57194619080
['57194619080', '57194619080', 2, 0]
Scopus Author ID53193 : 57194621940
['57194621940', '57194621940', 1, 0]
Scopus Author ID53194 : 7102870007
['7102870007', '7102870007', 295, 16]
Scopus Author ID53195 : 25228815200
['25228815200', '25228815200', 53, 25]
Scopu

['7403347843', '7403347843', 21, 22]
Scopus Author ID53294 : 54406138700
['54406138700', '54406138700', 7, 23]
Scopus Author ID53295 : 56909539600
['56909539600', '56909539600', 48, 11]
Scopus Author ID53296 : 57159761700
['57159761700', '57159761700', 1, 0]
Scopus Author ID53297 : 7003541712
['7003541712', '7003541712', 113, 23]
Scopus Author ID53298 : 7102796009
['7102796009', '7102796009', 30, 11]
Scopus Author ID53299 : 7201614399
['7201614399', '7201614399', 27, 23]
Scopus Author ID53300 : 7601565282
['7601565282', '7601565282', 61, 23]
Scopus Author ID53301 : 10045951100
['10045951100', '10045951100', 169, 31]
Scopus Author ID53302 : 27171673900
['27171673900', '27171673900', 78, 31]
Scopus Author ID53303 : 56406323600
['56406323600', '56406323600', 13, 25]
Scopus Author ID53304 : 57089784100
['57089784100', '57089784100', 3, 0]
Scopus Author ID53305 : 57198644478
['57198644478', '57198644478', 100, 31]
Scopus Author ID53306 : 7202339478
['7202339478', '7202339478', 195, 31]
Scop

['55725020900', '55725020900', 77, 19]
Scopus Author ID53405 : 56949974500
['56949974500', '56949974500', 2, 0]
Scopus Author ID53406 : 57102106700
['57102106700', '57102106700', 18, 23]
Scopus Author ID53407 : 57102680400
['57102680400', '57102680400', 1, 0]
Scopus Author ID53408 : 6603177519
['6603177519', '6603177519', 53, 16]
Scopus Author ID53409 : 7005832773
['7005832773', '7005832773', 34, 19]
Scopus Author ID53410 : 7402516297
['7402516297', '7402516297', 271, 23]
Scopus Author ID53411 : 36644583400
['36644583400', '36644583400', 33, 27]
Scopus Author ID53412 : 56024272300
['56024272300', '56024272300', 39, 27]
Scopus Author ID53413 : 56519356700
['56519356700', '56519356700', 16, 27]
Scopus Author ID53414 : 57109970600
['57109970600', '57109970600', 6, 27]
Scopus Author ID53415 : 57110010700
['57110010700', '57110010700', 6, 27]
Scopus Author ID53416 : 57110041200
['57110041200', '57110041200', 4, 0]
Scopus Author ID53417 : 7003969189
['7003969189', '7003969189', 150, 27]
Scop

['56683744900', '56683744900', 19, 13]
Scopus Author ID53517 : 7103260915
['7103260915', '7103260915', 106, 13]
Scopus Author ID53518 : 8239460500
['8239460500', '8239460500', 22, 13]
Scopus Author ID53519 : 8983983300
['8983983300', '8983983300', 44, 13]
Scopus Author ID53520 : 24330871300
['24330871300', '24330871300', 15, 13]
Scopus Author ID53521 : 56457408600
['56457408600', '56457408600', 5, 27]
Scopus Author ID53522 : 55471328100
['55471328100', '55471328100', 8, 27]
Scopus Author ID53523 : 7005625709
['7005625709', '7005625709', 19, 27]
Scopus Author ID53524 : 35243171500
['35243171500', '35243171500', 86, 25]
Scopus Author ID53525 : 55891489600
['55891489600', '55891489600', 10, 13]
Scopus Author ID53526 : 56322778800
['56322778800', '56322778800', 6, 25]
Scopus Author ID53527 : 57192168551
['57192168551', '57192168551', 1, 0]
Scopus Author ID53528 : 57192173476
['57192173476', '57192173476', 3, 0]
Scopus Author ID53529 : 6701589868
['6701589868', '6701589868', 73, 13]
Scopus 

['54879788400', '54879788400', 3, 0]
Scopus Author ID53628 : 55587954800
['55587954800', '55587954800', 10, 25]
Scopus Author ID53629 : 56275183800
['56275183800', '56275183800', 275, 25]
Scopus Author ID53630 : 57189260068
['57189260068', '57189260068', 1, 0]
Scopus Author ID53631 : 57189270384
['57189270384', '57189270384', 1, 0]
Scopus Author ID53632 : 7102606509
['7102606509', '7102606509', 193, 25]
Scopus Author ID53633 : 7408220900
['7408220900', '7408220900', 125, 25]
Scopus Author ID53634 : 56140544300
['56140544300', '56140544300', 20, 13]
Scopus Author ID53635 : 7404517372
['7404517372', '7404517372', 104, 13]
Scopus Author ID53636 : 13404239500
['13404239500', '13404239500', 18, 19]
Scopus Author ID53637 : 35329672300
['35329672300', '35329672300', 38, 19]
Scopus Author ID53638 : 43561856700
['43561856700', '43561856700', 22, 19]
Scopus Author ID53639 : 6508260037
['6508260037', '6508260037', 104, 19]
Scopus Author ID53640 : 6701821355
['6701821355', '6701821355', 87, 19]
Sc